step2 데이터 읽어오기  
step3 데이터 정제    
토큰화  했을때 토큰의 개수가 15개 넘어가는 문장을 학습데이터에서 제외시킨다  
step4 평가 데이터셋 분리  
단어장 크기는 12,000이상으로 설정하고 데이터의 20퍼센트는 평가 데이터셋으로 사용   
step5 인공지능 만들기  
모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계하세요! (Loss는 아래 제시된 Loss 함수를 그대로 사용!)

## step2 데이터 읽어오기

In [1]:
import os, re 
import numpy as np
import tensorflow as tf
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])
print('ok')

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]
ok


## step3 데이터 정제

In [2]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence
print('ok')

ok


In [3]:
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)
        
print('ok')

ok


In [4]:
def tokenize(corpus):
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    tokenizer.fit_on_texts(corpus)
    tensor = tokenizer.texts_to_sequences(corpus)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post',maxlen =15)  
    
    print(tensor, tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)
print('ok')

[[   2   50    5 ...    0    0    0]
 [   2   17 2639 ...    0    0    0]
 [   2   36    7 ...   43    3    0]
 ...
 [   5   22    9 ...   10 1013    3]
 [  37   15 9049 ...  877  647    3]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fbeb435b190>
ok


In [5]:
src_input = tensor[:, :-1] 
tgt_input = tensor[:, 1:]    
print('ok')

ok


In [6]:
len(src_input)
print('ok')

ok


In [7]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE
VOCAB_SIZE = tokenizer.num_words + 1   
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset
print('ok')

ok


##  step4 평가 데이터셋 분리

In [8]:
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, tgt_input, test_size=0.2, random_state=20)
print("Source Train:", enc_train.shape)  
print("Target Train:", dec_train.shape)  
print('ok')

Source Train: (140599, 14)
Target Train: (140599, 14)
ok


## step5 인공지능 만들기

In [9]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)
print('ok')

ok


In [10]:

for src_sample, tgt_sample in dataset.take(1): break

# 한 배치만 불러온 데이터를 모델에 넣어봅니다
model(src_sample)
print('ok')

ok


In [11]:
model.summary()
print('ok')

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12301025  
Total params: 29,012,961
Trainable params: 29,012,961
Non-trainable params: 0
_________________________________________________________________
ok


In [12]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
686/686 [==============================] - 128s 183ms/step - loss: 3.6404
Epoch 2/10
686/686 [==============================] - 125s 182ms/step - loss: 3.1502
Epoch 3/10
686/686 [==============================] - 125s 182ms/step - loss: 2.9396
Epoch 4/10
686/686 [==============================] - 125s 182ms/step - loss: 2.7763
Epoch 5/10
686/686 [==============================] - 125s 182ms/step - loss: 2.6366
Epoch 6/10
686/686 [==============================] - 125s 182ms/step - loss: 2.5118
Epoch 7/10
686/686 [==============================] - 125s 182ms/step - loss: 2.3961
Epoch 8/10
686/686 [==============================] - 125s 182ms/step - loss: 2.2891
Epoch 9/10
686/686 [==============================] - 125s 182ms/step - loss: 2.1883
Epoch 10/10
686/686 [==============================] - 125s 182ms/step - loss: 2.0931


In [27]:
def generate_text(model, tokenizer, init_sentence="<start>hello", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [28]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love you , liberian girl <end> '

## 회고
- 이번 프로젝트에서 **어려웠던 점**: 자연어 처리는 처음 해 보는 부분이라 너무 생소해서 프로젝트에 대한 전체적인 애해 과정에서 힘들었습니다.

- 프로젝트를 진행하면서 **알아낸 점** 혹은 **아직 모호한 점**.: batch size , epoch, buffer size 의 정확한 의미에 대해서 이해 하였습니다.

- 루브릭 평가 지표를 맞추기 위해 **시도한 것들** :사실 노력한 부분은 없습니다. epoch 를 10회로 설정하였고 학습 과정에서 저절로 loss 가 줄어들었습니다.
- **자기 다짐** : 이떄까지 프로젝트는 사진을 활용한 딥러닝 프로그램을 진행하였는데, 자연어 처리인 프로젝트는 처음 진행하여서 생소하였고 프로젝트 진행하는 과정에서 어려움을 느끼었습니다. 하지만 딥러닝 작동의 기본적인 구동 원리는 비슷하여서 빨리 습득 할 수 있었습니다. 앞으로도 프로젝트, fundamental 을 더 꼼꼼하게 공부하고 복습을 주기적으로 해야겠습니다.
